fine tune temperature

In [1]:
# import modules
from deepvisualinsight.MMS import MMS
from deepvisualinsight.backend import get_alpha
import sys
import os
import time
from scipy.special import softmax
import torch
import numpy as np

In [18]:
# content_path = "E:\\DVI_exp_data\\resnet18_mnist"
# content_path = "E:\\DVI_exp_data\\adv_training\\PGD.resnet18_cifar10"
content_path = "E:\\DVI_exp_data\\inexpressive_model"
content_path = "E:\\DVI_exp_data\\resnet50_cifar10"
sys.path.append(content_path)

from Model.model import *
net = resnet50()
# net = CIFAR_17()
classes = ("airplane", "car", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck")

mms = MMS(content_path, net, 15, 15, 1, 512, 10, classes, temperature=.02, cmap="tab10", resolution=400, neurons=256,
          verbose=1, temporal=False, split=-1, advance_border_gen=True, attack_device="cuda:0")

Finish loading content!


In [19]:
# hyperparameters
EPOCH = 120

In [22]:
model_location = os.path.join(mms.model_path, "Epoch_{:d}".format(EPOCH), "subject_model.pth")
net.load_state_dict(torch.load(model_location, map_location=torch.device("cpu")))
net = net.to(mms.device)
net.eval()

repr_model = torch.nn.Sequential(*(list(net.children())[-1:]))
repr_model = repr_model.to(mms.device)
repr_model = repr_model.eval()

In [23]:
for param in repr_model.parameters():
    param.requires_grad = False

In [24]:
train_data = mms.get_epoch_train_repr_data(EPOCH)
border_points = mms.get_epoch_border_centers(EPOCH)
fitting_data = np.concatenate((train_data, border_points), axis=0)

In [28]:
alpha = get_alpha(repr_model, fitting_data, temperature=.3, device=torch.device("cuda:0"), verbose=1)

Gradients calculation: 58.00 seconds	softmax with temperature: 2.00 seconds


In [29]:
l = []
for i in range(len(alpha)):
    l.append(np.sum(alpha[i]>alpha[i].mean()))

In [30]:
max(l),min(l), np.mean(l)

(663, 482, 577.9708)

In [11]:
sys.path.remove(content_path)